In [1]:
import hashlib # importa a biblioteca que faz o calculo da hash
import json # importa o módulo que manipula arquivos json
from time import time # importa a função time para obter o timestamp (data e hora)

saldo_atual = 0  # Variável global para acompanhar o saldo atual

class Bloco:
    def __init__(self, index, dados, hash_anterior=''):
        # inicializador o bloco com os parametros fornecidos e os atributos necessarios
        self.index = index # indice do bloco da cadeia
        self.timestamp = time() # Registrar o momento de criação do bloco
        self.dados = dados # Dados armazenados no bloco (ex: transações)
        self.hash_anterior = hash_anterior # Hash do bloco anterior na cadeia
        self.nonce = 0 # Valor usado na prova de trabalho para alterar o hash
        self.hash_atual = self.gerar_hash() # Gerar o hash do bloco

    def gerar_hash(self):
        # Função para gerar o hash SHA-256 do bloco
        conteudo_bloco = json.dumps({
            'index': self.index,
            'timestamp': self.timestamp,
            'dados': self.dados,
            'hash_anterior': self.hash_anterior,
            'nonce': self.nonce,
        }, sort_keys=True).encode() # Converte os dados do bloco em uma string JSON ordenada
        return hashlib.sha256(conteudo_bloco).hexdigest() # Calcula o hash SHA-256 do conteúdo do bloco

    def prova_de_trabalho(self, dificuldade):
        # realiza a prova de trabalho ajustando o nonce até que o hash comece com 'dificuldade' zero
        while self.hash_atual[:dificuldade] != '0' * dificuldade:
            self.nonce += 1 # incrementa o nonce para alterar o hash
            self.hash_atual = self.gerar_hash() # recalcula o hash com novo nonce

class Blockchain:
    def __init__(self):
        # inicializa a blockchain com um bloco genesis e definir a dificuldade da prova de trabalho
        self.cadeia = [self.criar_bloco_genesis()] # Lista que armazena os blocos na cadeia
        self.dificuldade = 2 # define o numero de zeros necessarios no hash da prova de trabalho

    def criar_bloco_genesis(self):
        return Bloco(0, "Bloco Genesis", "0") # cria bloco genesis
    
    def obter_ultimo_bloco(self):
        # retorna o ultimo bloco da cadeia
        return self.cadeia[-1]
    
    def adicionar_bloco(self, novo_bloco):
        # adiciona um novo bloco à cadeia
        novo_bloco.hash_anterior = self.obter_ultimo_bloco().hash_atual # define o hash do bloco anterior
        novo_bloco.prova_de_trabalho(self.dificuldade) # realiza a prova de trabalho para alterar o hash
        self.cadeia.append(novo_bloco) # adiciona o novo bloco à cadeia

    def validar_cadeia(self):
        # valida a integridade da cadeia, validando hashes e encadeamento
        for i in range(1, len(self.cadeia)):
            bloco_atual = self.cadeia[i]
            bloco_anterior = self.cadeia[i-1]

        # verificar se o hash armazenado corresponde ao hash calculado
        if bloco_atual.hash_atual != bloco_atual.gerar_hash():
            return False # retorna false se houver discrepancia
        
        # verificar se o hash do bloco anterior correponde ao hash armazenado no bloco anterior
        if bloco_anterior.hash_atual != bloco_anterior.gerar_hash():
            return False # retorna false se houver discrepancia

        return True # retorna true se a cadeia for valida

    def salvar_em_json(self, nome_arquivo='blockchain.json'):
        # salvar a cadeia em um arquivo JSON
        with open(nome_arquivo, 'w') as arquivo:
            json.dump([bloco.__dict__ for bloco in self.cadeia], arquivo, indent=4)

# função para coletar dados de exames medicos
def calcular_saldo_carteira(saldo_atual):
    valor = float(input("Digite a alteração de valor (positivo para entrada, negativo para saída):\n> "))
    saldo_atualizado = saldo_atual + valor
    return {
        'entrada': valor,
        'saldo_anterior': saldo_atual,
        'saldo_atual': saldo_atualizado
    } # retorna um dicionario com os dados coletados

In [2]:
blockchain = Blockchain()

In [4]:
# Coletar os dados da cateira
saldo_carteira = calcular_saldo_carteira(saldo_atual)
saldo_atual = saldo_carteira['saldo_atual']  # Atualiza o saldo global

# Cria um novo bloco com os dados coletados
novo_bloco = Bloco(blockchain.obter_ultimo_bloco().index + 1, saldo_carteira)
# define o indice do novo bloco como indice do ultimo bloco + 1
# associa os dados coletados ao bloco criado

# adiciona o novo bloco a blockchain
blockchain.adicionar_bloco(novo_bloco)

# verificar intergidade da blockchain apos adicao do bloco
if blockchain.validar_cadeia():
    blockchain.salvar_em_json()
    print("A cadeia é válida.")
else:
    print("A cadeia é inválida.")

A cadeia é válida.


In [ ]:
def print_json_content(filename):
    # tenta abrir e processar o arquivo json
    try:
        with open(filename, 'r') as file:
            data = json.load(file)
            print(json.dumps(data, indent=4))
    except FileNotFoundError:
        print(f"Erro: Arquivo {filename} não encontrado")
    except json.JSONDecodeError:
        print(f"Erro: Arquivo {filename} ao ler arquivo")